In [2]:
from tensorflow.python.estimator import keras
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle

# 데이터셋 준비
train_df = pd.read_csv('../data/SisFall_train_L.csv')  # 훈련 파일
test_df = pd.read_csv('../data/SisFall_val_L.csv')  # 테스트 파일

X_train = []  # 훈련 입력 데이터
y_train = []  # 훈련 출력 데이터

X_test = []  # 테스트 입력 데이터
y_test = []  # 테스트 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(train_df) - 39, 40):
    # gx, gy, gz 값 가져오기
    gx_values = train_df.loc[i:i+39, 'gx'].values
    gy_values = train_df.loc[i:i+39, 'gy'].values
    gz_values = train_df.loc[i:i+39, 'gz'].values

    # 레이블 값 가져오기
    labels = train_df.loc[i:i+39, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_train.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_train.append(labels[0])

for i in range(0, len(test_df) - 39, 40):
    # gx, gy, gz 값 가져오기
    gx_values = test_df.loc[i:i+39, 'gx'].values
    gy_values = test_df.loc[i:i+39, 'gy'].values
    gz_values = test_df.loc[i:i+39, 'gz'].values

    # 레이블 값 가져오기
    labels = test_df.loc[i:i+39, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_test.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_test.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded_train = label_encoder.fit_transform(y_train)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoder_train = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder_train.fit_transform(integer_encoded_train)

integer_encoded_test = label_encoder.transform(y_test)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoder_test = OneHotEncoder(sparse_output=False)
y_test = onehot_encoder_test.fit_transform(integer_encoded_test)

# CNN-LSTM 모델 정의
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(40, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델의 가중치를 피클 파일로 저장
best_model_weights = history.model.get_weights()
pickle.dump(best_model_weights, open('../model/best_model_weights_cnn_lstm.pkl', 'wb'))

# 레이블 인코더의 클래스 정보를 저장
np.save('../model/label_encoder_classes_cnn_lstm.npy', label_encoder.classes_)

# 가중치를 로드하기 위해 모델 구성
loaded_model = keras.models.Sequential()
loaded_model.add(keras.layers.LSTM(128, input_shape=(3, 40)))  # 3개의 시퀀스, 각 시퀀스에 40개의 피처
loaded_model.add(keras.layers.Dense(7, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 피클 파일에서 가중치 로드
loaded_model_weights = pickle.load(open('../model/best_model_weights_cnn_lstm.pkl', 'rb'))

# 모델에 로드된 가중치 설정
loaded_model.set_weights(loaded_model_weights)

# 모델 컴파일
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 저장
loaded_model.save('../model/loaded_model_cnn_lstm.h5')

Epoch 1/30
214/214 [==============================] - 5s 14ms/step - loss: 0.7662 - accuracy: 0.6746 - val_loss: 0.7295 - val_accuracy: 0.7095
Epoch 2/30
214/214 [==============================] - 3s 13ms/step - loss: 0.5488 - accuracy: 0.7718 - val_loss: 0.6731 - val_accuracy: 0.7399
Epoch 3/30
214/214 [==============================] - 3s 13ms/step - loss: 0.4601 - accuracy: 0.8117 - val_loss: 0.6465 - val_accuracy: 0.7580
Epoch 4/30
214/214 [==============================] - 3s 13ms/step - loss: 0.4055 - accuracy: 0.8327 - val_loss: 0.5859 - val_accuracy: 0.7615
Epoch 5/30
214/214 [==============================] - 3s 13ms/step - loss: 0.3652 - accuracy: 0.8510 - val_loss: 0.6465 - val_accuracy: 0.7434
Epoch 6/30
214/214 [==============================] - 3s 13ms/step - loss: 0.3275 - accuracy: 0.8684 - val_loss: 0.5771 - val_accuracy: 0.7697
Epoch 7/30
214/214 [==============================] - 3s 13ms/step - loss: 0.3029 - accuracy: 0.8755 - val_loss: 0.6217 - val_accuracy: 0.7592

KeyboardInterrupt: 

In [3]:
import matplotlib.pyplot as plt

# Get the training history
train_loss = history.history['loss']
train_acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.ylim([0, 1])
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.ylim([0, 1])
plt.show()

NameError: name 'history' is not defined

In [5]:
from keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import pickle

# 데이터셋 준비
train_df = pd.read_csv('../data/SisFall_train_L_20.csv')  # 훈련 파일
test_df = pd.read_csv('../data/SisFall_val_L_20.csv')  # 테스트 파일

X_train = []  # 훈련 입력 데이터
y_train = []  # 훈련 출력 데이터

X_test = []  # 테스트 입력 데이터
y_test = []  # 테스트 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(train_df) - 19, 20):
    # gx, gy, gz 값 가져오기
    gx_values = train_df.loc[i:i+19, 'gx'].values
    gy_values = train_df.loc[i:i+19, 'gy'].values
    gz_values = train_df.loc[i:i+19, 'gz'].values

    # 레이블 값 가져오기
    labels = train_df.loc[i:i+19, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_train.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_train.append(labels[0])

for i in range(0, len(test_df) - 19, 20):
    # gx, gy, gz 값 가져오기
    gx_values = test_df.loc[i:i+19, 'gx'].values
    gy_values = test_df.loc[i:i+19, 'gy'].values
    gz_values = test_df.loc[i:i+19, 'gz'].values

    # 레이블 값 가져오기
    labels = test_df.loc[i:i+19, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_test.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_test.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded_train = label_encoder.fit_transform(y_train)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoder_train = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder_train.fit_transform(integer_encoded_train)

integer_encoded_test = label_encoder.transform(y_test)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoder_test = OneHotEncoder(sparse_output=False)
y_test = onehot_encoder_test.fit_transform(integer_encoded_test)

# CNN-LSTM 모델 정의
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(20, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델의 가중치를 피클 파일로 저장
best_model_weights = model.get_weights()
pickle.dump(best_model_weights, open('../model/best_model_weights_cnn_lstm.pkl', 'wb'))

# 레이블 인코더의 클래스 정보를 저장
np.save('../model/label_encoder_classes_cnn_lstm.npy', label_encoder.classes_)

# 모델 저장
model.save('../model/loaded_model_cnn_lstm.h5')

Epoch 1/30
214/214 [==============================] - 4s 12ms/step - loss: 0.8720 - accuracy: 0.6423 - val_loss: 0.7253 - val_accuracy: 0.7130
Epoch 2/30
214/214 [==============================] - 2s 8ms/step - loss: 0.6764 - accuracy: 0.7246 - val_loss: 0.6628 - val_accuracy: 0.7300
Epoch 3/30
214/214 [==============================] - 2s 8ms/step - loss: 0.6167 - accuracy: 0.7488 - val_loss: 0.6908 - val_accuracy: 0.7113
Epoch 4/30
214/214 [==============================] - 2s 8ms/step - loss: 0.5786 - accuracy: 0.7635 - val_loss: 0.6763 - val_accuracy: 0.7376
Epoch 5/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5500 - accuracy: 0.7767 - val_loss: 0.6553 - val_accuracy: 0.7434
Epoch 6/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5246 - accuracy: 0.7919 - val_loss: 0.7048 - val_accuracy: 0.7434
Epoch 7/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5046 - accuracy: 0.7977 - val_loss: 0.5991 - val_accuracy: 0.7592
Epoch

In [6]:
from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import pickle

# 데이터셋 준비
train_df = pd.read_csv('../data/SisFall_train_L_20.csv')  # 훈련 파일
test_df = pd.read_csv('../data/SisFall_val_L_20.csv')  # 테스트 파일

X_train = []  # 훈련 입력 데이터
y_train = []  # 훈련 출력 데이터

X_test = []  # 테스트 입력 데이터
y_test = []  # 테스트 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(train_df) - 19, 20):
    # gx, gy, gz 값 가져오기
    gx_values = train_df.loc[i:i+19, 'gx'].values
    gy_values = train_df.loc[i:i+19, 'gy'].values
    gz_values = train_df.loc[i:i+19, 'gz'].values

    # 레이블 값 가져오기
    labels = train_df.loc[i:i+19, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_train.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_train.append(labels[0])

for i in range(0, len(test_df) - 19, 20):
    # gx, gy, gz 값 가져오기
    gx_values = test_df.loc[i:i+19, 'gx'].values
    gy_values = test_df.loc[i:i+19, 'gy'].values
    gz_values = test_df.loc[i:i+19, 'gz'].values

    # 레이블 값 가져오기
    labels = test_df.loc[i:i+19, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_test.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_test.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded_train = label_encoder.fit_transform(y_train)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoder_train = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder_train.fit_transform(integer_encoded_train)

integer_encoded_test = label_encoder.transform(y_test)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoder_test = OneHotEncoder(sparse_output=False)
y_test = onehot_encoder_test.fit_transform(integer_encoded_test)

# CNN-LSTM 모델 정의
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(20, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))  # Dropout layer with 20% dropout rate
model.add(BatchNormalization())  # Batch normalization layer
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델의 가중치를 피클 파일로 저장
best_model_weights = model.get_weights()
pickle.dump(best_model_weights, open('../model/best_model_weights_cnn_lstm.pkl', 'wb'))

# 레이블 인코더의 클래스 정보를 저장
np.save('../model/label_encoder_classes_cnn_lstm.npy', label_encoder.classes_)

# 모델 저장
model.save('../model/loaded_model_cnn_lstm.h5')

Epoch 1/30
214/214 [==============================] - 5s 11ms/step - loss: 0.8817 - accuracy: 0.6232 - val_loss: 0.7833 - val_accuracy: 0.7043
Epoch 2/30
214/214 [==============================] - 2s 9ms/step - loss: 0.6770 - accuracy: 0.7037 - val_loss: 0.6217 - val_accuracy: 0.7148
Epoch 3/30
214/214 [==============================] - 2s 9ms/step - loss: 0.6218 - accuracy: 0.7237 - val_loss: 0.6478 - val_accuracy: 0.7236
Epoch 4/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5872 - accuracy: 0.7355 - val_loss: 0.6316 - val_accuracy: 0.7019
Epoch 5/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5559 - accuracy: 0.7465 - val_loss: 0.6731 - val_accuracy: 0.7276
Epoch 6/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5347 - accuracy: 0.7611 - val_loss: 0.6422 - val_accuracy: 0.7119
Epoch 7/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5151 - accuracy: 0.7678 - val_loss: 0.6851 - val_accuracy: 0.6926
Epoch

In [7]:
from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import pickle

# 데이터셋 준비
train_df = pd.read_csv('../data/SisFall_train_L_20.csv')  # 훈련 파일
test_df = pd.read_csv('../data/SisFall_val_L_20.csv')  # 테스트 파일

X_train = []  # 훈련 입력 데이터
y_train = []  # 훈련 출력 데이터

X_test = []  # 테스트 입력 데이터
y_test = []  # 테스트 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(train_df) - 19, 20):
    # gx, gy, gz 값 가져오기
    gx_values = train_df.loc[i:i+19, 'gx'].values
    gy_values = train_df.loc[i:i+19, 'gy'].values
    gz_values = train_df.loc[i:i+19, 'gz'].values

    # 레이블 값 가져오기
    labels = train_df.loc[i:i+19, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_train.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_train.append(labels[0])

for i in range(0, len(test_df) - 19, 20):
    # gx, gy, gz 값 가져오기
    gx_values = test_df.loc[i:i+19, 'gx'].values
    gy_values = test_df.loc[i:i+19, 'gy'].values
    gz_values = test_df.loc[i:i+19, 'gz'].values

    # 레이블 값 가져오기
    labels = test_df.loc[i:i+19, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X_test.append(np.transpose([gx_values, gy_values, gz_values]))

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y_test.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded_train = label_encoder.fit_transform(y_train)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoder_train = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder_train.fit_transform(integer_encoded_train)

integer_encoded_test = label_encoder.transform(y_test)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoder_test = OneHotEncoder(sparse_output=False)
y_test = onehot_encoder_test.fit_transform(integer_encoded_test)

# CNN-LSTM 모델 정의
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(20, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))  # Dropout layer with 20% dropout rate
model.add(BatchNormalization())  # Batch normalization layer
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델의 가중치를 피클 파일로 저장
best_model_weights = model.get_weights()
pickle.dump(best_model_weights, open('../model/best_model_weights_cnn_lstm.pkl', 'wb'))

# 레이블 인코더의 클래스 정보를 저장
np.save('../model/label_encoder_classes_cnn_lstm.npy', label_encoder.classes_)

# 모델 저장
model.save('../model/loaded_model_cnn_lstm.h5')

Epoch 1/30
214/214 [==============================] - 5s 12ms/step - loss: 0.8670 - accuracy: 0.6299 - val_loss: 0.6914 - val_accuracy: 0.7101
Epoch 2/30
214/214 [==============================] - 2s 9ms/step - loss: 0.6705 - accuracy: 0.7047 - val_loss: 0.6365 - val_accuracy: 0.7148
Epoch 3/30
214/214 [==============================] - 2s 10ms/step - loss: 0.6186 - accuracy: 0.7249 - val_loss: 0.6541 - val_accuracy: 0.6990
Epoch 4/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5882 - accuracy: 0.7353 - val_loss: 0.6942 - val_accuracy: 0.7013
Epoch 5/30
214/214 [==============================] - 2s 10ms/step - loss: 0.5546 - accuracy: 0.7493 - val_loss: 0.6405 - val_accuracy: 0.7002
Epoch 6/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5408 - accuracy: 0.7553 - val_loss: 0.6386 - val_accuracy: 0.7165
Epoch 7/30
214/214 [==============================] - 2s 9ms/step - loss: 0.5181 - accuracy: 0.7638 - val_loss: 0.6432 - val_accuracy: 0.7160
Epo